In [1]:
import numpy as np
import pandas as pd

import data_gen_read
import dataloader_build
import model_structure
import model_train
import model_test
import model_run

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

##### Read data/generate data

In [9]:
sequence_length = 3
# data = read_data(datatype = 1, num_data = 500, step = 1, func = "Linear", pattern = 1)
data = data_gen_read.read_data(datatype = 2, dataset = "EVcharging_data_univariate_p.csv")

train_dataloader, val_dataloader, test_dataloader = dataloader_build.build_data_loader(data, seq_length=sequence_length, batch_size=1)

In [4]:
dataloader_build.show_data_loader(train_dataloader)

inputs:  tensor([[[   0.0000,   65.9358],
         [  21.0000,   37.0784],
         [  43.0000,   31.3078]],

        [[  21.0000,   37.0784],
         [  43.0000,   31.3078],
         [  65.0000,   40.6731]],

        [[  43.0000,   31.3078],
         [  65.0000,   40.6731],
         [  85.0000,   55.7328]],

        [[  65.0000,   40.6731],
         [  85.0000,   55.7328],
         [ 105.0000,   25.5692]],

        [[  85.0000,   55.7328],
         [ 105.0000,   25.5692],
         [ 125.0000,   26.1922]],

        [[ 105.0000,   25.5692],
         [ 125.0000,   26.1922],
         [ 147.0000,   40.3305]],

        [[ 125.0000,   26.1922],
         [ 147.0000,   40.3305],
         [ 169.0000,   23.3168]],

        [[ 147.0000,   40.3305],
         [ 169.0000,   23.3168],
         [ 189.0000,   50.4057]],

        [[ 169.0000,   23.3168],
         [ 189.0000,   50.4057],
         [ 211.0000,   28.3316]],

        [[ 189.0000,   50.4057],
         [ 211.0000,   28.3316],
         [ 232.0

#### Data trend visualization
##### change them into a function

In [6]:
## Change 
import altair as alt

source = pd.DataFrame({
    'x': np.arange(0,len(data)),
    'f(x)': data
  })

alt.Chart(source).mark_line().encode(
      x='x',
      y='f(x)'
  )

#### Model bundle

In [10]:
train_losses, val_losses, test_losses = model_run.model_bundle(train_dataloader = train_dataloader, val_dataloader=val_dataloader, test_dataloader = test_dataloader, device = device, seq_len=sequence_length, model_type=1, num_epoch=10, verbose=True)

+--------------------+
|Print the linear model summary|
+--------------------+
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 1]               4
Total params: 4
Trainable params: 4
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------
None


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x6 and 3x1)

#### Result diagnoses

#### Loss function visualization

In [ ]:
def quick_loss_plot(train_losses, val_losses, model_type = 4, loss_type = "MSE Loss", sparse_n = 0):
    '''
    For each train/test loss trajectory, plot loss by epoch
    '''
    model_dict = {
        1: "Linear",
        2: "MLP",
        3: "CNN",
        4: "LSTM"
    }

    data_label_list = [(train_losses, val_losses, model_dict[model_type])]

    for i,(train_data,val_data,label) in enumerate(data_label_list):    
        plt.plot(train_data,linestyle='--',color=f"C{i}", label=f"{label} Train")
        plt.plot(val_data,color=f"C{i}", label=f"{label} Val",linewidth=3.0)

    plt.legend()
    plt.ylabel(loss_type)
    plt.xlabel("Epoch")
    plt.legend(bbox_to_anchor=(1,1),loc='upper left')
    plt.show()

In [ ]:
def quick_prediction_plot(data_label_list, sparse_n = 0):
    pass

In [ ]:
### Test with testing data
with torch.no_grad():
    # shift train predictions for plotting
    train_plot = np.ones_like(data) * np.nan
    y_pred = model(X_train)
    y_pred = y_pred[:, -1, :]
    train_plot[lookback:train_size] = model(X_train)[:, -1, :]
    # shift test predictions for plotting
    test_plot = np.ones_like(data) * np.nan
    test_plot[train_size+lookback:len(data)] = model(X_test)[:, -1, :]
# plot
plt.plot(data, c='b')
plt.plot(train_plot, c='r')
plt.plot(test_plot, c='g')
plt.show()